In [45]:
db.ai = db.armory_item.aggregate([
    {
        '$addFields':
            {
                '_id': '$item_id'
            }
    }
])

In [46]:
type(db.ai)

pymongo.command_cursor.CommandCursor

In [47]:
ai = db.armory_item.aggregate([
    {
        '$addFields':
            {
                '_id': '$item_id'
            }
    }
])

In [48]:
del db.ai

In [49]:
db.ai.insert_many([a for a in ai])

In [50]:
db.bi = db.armory_item

In [51]:
db.ai

Collection(Database(MongoClient(host=['jcs-lambda-shard-00-00-fc9ho.mongodb.net:27017', 'jcs-lambda-shard-00-02-fc9ho.mongodb.net:27017', 'jcs-lambda-shard-00-01-fc9ho.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='jcs-lambda-shard-0', ssl=True), 'rpg_db'), 'ai')

In [53]:
db.ai.rename('armory_item', dropTarget=True)

{'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1582251038, 1),
  'signature': {'hash': b'.\xf3W\xc57\x16\xf4\xa4\x12Ru\xf4n*\x80\x18\x02s\xff\x98',
   'keyId': 6794816602199031810}},
 'operationTime': Timestamp(1582251038, 1)}

In [54]:
def re_id(collection, real_id):
    if 'temp' in db.list_collection_names():
        db.drop_collection('temp')
    db.temp.insert_many(
        [
            doc for doc in db[collection].aggregate([
                {
                    '$addFields':
                        {
                            '_id': f'${real_id}'
                        }
                }
            ])
        ]
    )
    db.temp.rename(collection, dropTarget=True)

In [55]:
re_id('armory_weapon', 'item_ptr_id')

In [58]:
to_rename = \
{
    'auth_permission': 'id',
    'charactercreator_character': 'character_id',
    'charactercreator_character_inventory': 'id',
    'charactercreator_cleric': 'character_ptr_id',
    'charactercreator_fighter': 'character_ptr_id',
    'charactercreator_mage': 'character_ptr_id',
    'charactercreator_necromancer': 'mage_ptr_id',
    'charactercreator_thief': 'character_ptr_id',
    'django_content_type': 'id',
    'django_migrations': 'id',
}

In [60]:
for col, rid in to_rename.items():
    print(col, rid)
    re_id(col, rid)

auth_permission id
charactercreator_character character_id
charactercreator_character_inventory id
charactercreator_cleric character_ptr_id
charactercreator_fighter character_ptr_id
charactercreator_mage character_ptr_id
charactercreator_necromancer mage_ptr_id
charactercreator_thief character_ptr_id
django_content_type id
django_migrations id


In [63]:
db.armory_item_weapon.insert_many([
    doc for doc in db.armory_item.aggregate([
        {
            '$lookup':
                {
                    'from': 'armory_weapon',
                    'localField': 'item_id',
                    'foreignField': 'item_ptr_id',
                    'as': 'weapon'
                }
        }
    ])
])

In [65]:
db.armory_item_weapon.count_documents({
    'weapon': {
        '$size': 0
    }
})

137